In [1]:
# requests는 작은 웹브라우저로 웹사이트 내용을 가져온다 
import requests
# BeautifulSoup을 통해 읽어온 웹페이지를 파싱한다
from bs4 import BeautifulSoup as bs
# random은 시간차를 두고 가져오기 위해 사용한다
import random
# time으로 시간을 구한다
import time
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러온다.
import pandas as pd

In [2]:
# 크롤링 할 사이트
base_url = 'https://www.bikeseoul.com/app/station/moveStationSearchView.do?currentPageNo='

# 전체 대여소 정보를 담아줄 비어있는 리스트를 만든다.
all_stations = []

def crawling_rent_station(pnum):
    response = requests.get( base_url + str(pnum))
        
    if response.status_code != 200:
        return False
    
    soup = bs(response.text, 'html.parser')
    tbody = soup.select('table.psboard1 > tbody')
    
    if not tbody:
        return False
    
    # response의 tbody 값을 전달해 각 대여소의 상세 정보를 받아온다.
    stations = station_info(tbody)
    # www.bikeseoul.com 사이트의 서버에 부담을 덜 주기 위해 시간차를 두고 가져온다.
    time.sleep(random.uniform(1,3))
    if stations:
        # 크롤링 해서 가져온 대여소 정보를 리스트에 담아 줍니다.
        all_stations.extend(stations)

        # 이전 결과가 있다면 페이지번호를 하나씩 더해서 재귀호출을 한다.
        pnum += 1
        # 몇 페이지를 크롤링 하고 있는지 찍어본다. 
        # 다 찍기엔 너무 많아서 10페이지에 한 번씩 출력하도록 한다.
        # 페이지 수를 크롤링 하기 전에 미리 알고 크롤링을 한다면 tqdm을 사용하는 것이 좋다.
        # 2019년 8월 자전거 대여소 페이지수는 307개 이다.
        if pnum % 10 == 0 :
            print(pnum)

        # 같은 함수를 재귀호출해서 실행한다.
        return crawling_rent_station(pnum)
    else:
        return all_stations

In [3]:
def station_info(tbody):
    if tbody: 
        trs = tbody[0].find_all('tr')
        rent_stations = []
        for tr in trs:
            # 대여소 정보를 담아준다.
            info = []
            # 대여소명
            name = tr.select('td.pl10')[0].get_text(strip=True)
            if '.' in name:
                info.append(name.split('.')[0].strip())
                info.append(name.split('.')[-1].lstrip())
            else:
                # 대여소 정보가 없다면 결측치로 처리한다.
                info.append(pd.np.nan)
                info.append(name)
            # 운영여부
            info.append(tr.select('td.pl10')[1].get_text(strip=True))
            # 주소
            info.append(tr.select('td.mhid')[0].get_text(strip=True))
            # 위도, 경도
            geo = tr.find('a')['param-data'].split(',')
            info.append(geo[0])
            info.append(geo[1])
            rent_stations.append(info)
        return rent_stations
    else :
        return False

In [4]:
crawling_rent_station(304)

[['964',
  '북한산힐스테이트 7차아파트',
  '운영중',
  '서울특별시 은평구 통일로 796 북한산 힐스테이트7차',
  '37.61654282',
  '126.92512512'],
 ['965',
  '서울특별시 은평병원',
  '운영중',
  '서울특별시 은평구 백련산로 90 서울특별시 은평병원',
  '37.59347916',
  '126.92318726'],
 ['966',
  '서울혁신파크1',
  '운영중',
  '서울특별시 은평구 통일로 684 서울혁신파크1',
  '37.60903168',
  '126.93467712'],
 ['967',
  '서울혁신파크2',
  '운영중',
  '서울특별시 은평구 통일로 684 서울혁신파크2',
  '37.60890961',
  '126.93241882'],
 ['968',
  '은평뉴타운 상림마을 13단지',
  '운영중',
  '서울특별시 은평구 진관4로 48-50 은평뉴타운 상림마을 13단지',
  '37.64208984',
  '126.92941284'],
 ['969',
  '은평 지웰테라스',
  '운영중',
  '서울특별시 은평구 진관4로 78-38 은평 지웰테라스',
  '37.64385986',
  '126.93187714'],
 ['971',
  '역촌 센트레빌 아파트',
  '운영중',
  '서울특별시 은평구 갈현로3나길 23 역촌 센트레빌 아파트 107동 인근',
  '37.60232925',
  '126.90654755'],
 ['972', '수색역', '운영중', '서울특별시 은평구 수색로 261 수색역', '37.58216095', '126.89492798'],
 ['9996',
  '시설2',
  '운영중',
  '서울특별시 성동구 청계천로 540 서울시설공단 공공자전거 운영처',
  '0.00000000',
  '0.00000000'],
 ['99998',
  '상암단말정비',
  '운영중',
  '서울특별시 서대문구 연세로5나길 19 53-47',
  '-87.39

In [5]:
all_stations[1]

['965',
 '서울특별시 은평병원',
 '운영중',
 '서울특별시 은평구 백련산로 90 서울특별시 은평병원',
 '37.59347916',
 '126.92318726']

In [6]:
pd.DataFrame(all_stations).sample()

,0,1,2,3,4,5
4,968,은평뉴타운 상림마을 13단지,운영중,서울특별시 은평구 진관4로 48-50 은평뉴타운 상림마을 13단지,37.64208984,126.92941284


In [7]:
# 전체 코드를 돌리기 전에 다시 리스트를 비워준다.
all_stations = []
# 아래의 pnum에 1을 입력하면 1페이지부터 끝까지 가져온다.
# pnum에 306 을 306페이지부터 끝까지 가져온다.
# pnum = 1
pnum = 306
# 크롤링 함수를 호출한다.
crawling_rent_station(pnum)

[[nan, '위트콤', '운영중', '서울특별시 서초구 방배로 110 위트콤', '0.00000000', '0.00000000'],
 [nan,
  '위트콤공장',
  '운영중',
  '서울특별시 서초구 방배로 110 석교빌딩 4층',
  '0.00000000',
  '0.00000000']]

In [8]:
# 크롤링한 전체 결과를 프린트 한다.
all_stations[:2]

[[nan, '위트콤', '운영중', '서울특별시 서초구 방배로 110 위트콤', '0.00000000', '0.00000000'],
 [nan,
  '위트콤공장',
  '운영중',
  '서울특별시 서초구 방배로 110 석교빌딩 4층',
  '0.00000000',
  '0.00000000']]

In [9]:
header = ['대여소번호', '대여소', '상태', '주소', '위도', '경도']
df = pd.DataFrame.from_records(all_stations, columns = header)
df.shape

(2, 6)

In [10]:
df.head()

,대여소번호,대여소,상태,주소,위도,경도
0,NaN,위트콤,운영중,서울특별시 서초구 방배로 110 위트콤,0.00000000,0.00000000
1,NaN,위트콤공장,운영중,서울특별시 서초구 방배로 110 석교빌딩 4층,0.00000000,0.00000000


In [11]:
df.tail()

,대여소번호,대여소,상태,주소,위도,경도
0,NaN,위트콤,운영중,서울특별시 서초구 방배로 110 위트콤,0.00000000,0.00000000
1,NaN,위트콤공장,운영중,서울특별시 서초구 방배로 110 석교빌딩 4층,0.00000000,0.00000000


In [12]:
df.to_csv('bike_rent_station.csv', index=False)

In [13]:
# 파일이 제대로 생성되었는지 확인
pd.read_csv('bike_rent_station.csv').head()

,대여소번호,대여소,상태,주소,위도,경도
0,NaN,위트콤,운영중,서울특별시 서초구 방배로 110 위트콤,0.0,0.0
1,NaN,위트콤공장,운영중,서울특별시 서초구 방배로 110 석교빌딩 4층,0.0,0.0
